<a href="https://colab.research.google.com/github/ryanhao1115/ML-for-Fraud-Detection/blob/main/5_2_DL_onehotencoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [217]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [294]:
!pip install imbalanced-learn

## Deep Learning model with One hot encoding data
1. import new return label data.
2. Under sampling data to address imbalance class issue.
4. Train model
5. Eval model
6. Create Risk-heatmap

In [296]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.backend import clear_session
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import tensorflow as tf
import random
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


In [219]:
def reset_seeds():
    '''
    reset random seeds for modeling
    '''
    np.random.seed(1)
    random.seed(2)
    if tf.__version__[0] == '2':
        tf.random.set_seed(3)
    else:
        tf.set_random_seed(3)
    print("RANDOM SEEDS RESET")

In [5]:
## import dataset 
path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
file = path + 'sales_clean.csv'
df = pd.read_csv(file)

In [6]:
df = df.drop(columns='Unnamed: 0')

In [7]:
df.nunique()

distributor        639
sales              209
branch              28
inv_type             3
invoice_no       30721
product_no        1094
prod_cla             7
qty                202
total_amt        16112
sale_price        8598
ship_qty           202
cust_type           20
return               2
discount_app         2
list_price        6720
inv_ship_days      133
dtype: int64

## import return label data

In [8]:
## import fraud data
file = path + 'frauds.csv'
df_fraud = pd.read_csv(file,header=None)
df_fraud.columns = ['invoice_no']

In [9]:
## import return new labeled data
file = path + 'return.csv'
df_fraud2 = pd.read_csv(file,header=None)
df_fraud2.columns = ['invoice_no']

In [10]:
len(df_fraud2)

24

In [11]:
def label_fraud(df, df_fraud, df_fraud2):
  df['fraud'] = np.zeros(len(df))
  frauds_l = df_fraud['invoice_no'].to_list()
  frauds_l2 = df_fraud2['invoice_no'].to_list()
  df.loc[df['invoice_no'].isin(frauds_l),'fraud'] = 1
  df.loc[df['invoice_no'].isin(frauds_l2),'fraud'] = 2
  return df

In [12]:
df = label_fraud(df, df_fraud,df_fraud2)

In [13]:
df['fraud'].value_counts()

0.0    228014
2.0       433
1.0       154
Name: fraud, dtype: int64

## Encoding

In [247]:
df.nunique()

distributor        639
sales              209
branch              28
inv_type             3
invoice_no       30721
product_no        1094
prod_cla             7
qty                202
total_amt        16112
sale_price        8598
ship_qty           202
cust_type           20
return               2
discount_app         2
list_price        6720
inv_ship_days      133
fraud                3
dtype: int64

In [38]:
cols = df.columns.to_list()
cols = ['distributor', 'sales', 'branch', 'inv_type', 'invoice_no', 'product_no', 'prod_cla', 'cust_type']

In [248]:
## remove some fields with many classes
cols1 = ['distributor', 'sales', 'invoice_no', 'product_no']
df_less = df.drop(columns=cols1)

In [249]:
def field_onehotencoding(df):
  cols = [ 'branch', 'inv_type',  'prod_cla', 'cust_type']
  df[cols] = df[cols].astype('str') 
  df_enc = pd.get_dummies(df)
  return df_enc

In [250]:
df_less = field_onehotencoding(df_less)

In [251]:
df_less.shape

(228601, 67)

## Resampling to address the imbalance class
Because fraud labels are given to invoice level, need to keep records of same invoice together. 

In [262]:
df_fraud = df_less[df_less['fraud'] == 1]
df_non_fraud = df_less[df_less['fraud'] == 0]
df_fraud2 = df_less[df_less['fraud'] == 2]

In [264]:
len(df_fraud2)

433

In [297]:
df_non_fraud.shape

(228014, 67)

In [267]:
def sampling_record(df_non_fraud,df_fraud,df_fraud2):
  '''
  Random sampling from unlabled records.
  concat with labeled records
  '''
  df = df_non_fraud.sample(n=30000,replace=False)
  print(len(df))

  ## Build training dataset
  df_train = df.iloc[:22000,:]
  df_train = pd.concat([df_train,df_fraud2])
  df_train.loc[df_train['fraud'] == 2, 'fraud'] = 1 
  df_train = df_train.sort_index()
  print(len(df_train))

  ## Build eval dataset
  df_test = df.iloc[22000:,:]
  df_test = pd.concat([df_test,df_fraud])
  df_test = df_test.sort_index()

  return df_train, df_test


In [268]:
df_train, df_test = sampling_record(df_non_fraud, df_fraud, df_fraud2)

30000
22433


In [269]:
df_train.shape

(22433, 67)

In [270]:
df_test.shape

(8154, 67)

In [271]:
df_train.nunique()

qty                 98
total_amt         3929
sale_price        2947
ship_qty            98
return               2
                  ... 
cust_type_KX01       2
cust_type_LX01       2
cust_type_MX01       2
cust_type_NX01       2
cust_type_ZZ01       2
Length: 67, dtype: int64

## Standardscale data

In [45]:
scaler = StandardScaler()

In [272]:
y_train = df_train['fraud'].values
X_train = df_train.drop(columns=['fraud'])

X_train = scaler.fit_transform(X_train)
y_train = y_train.reshape(len(y_train),1)

In [273]:
y_test = df_test['fraud'].values
X_test = df_test.drop(columns=['fraud'])

X_test = scaler.transform(X_test)
y_test = y_test.reshape(len(y_test),1)

In [276]:
X_train.shape

(22433, 66)

In [274]:
y_test.shape

(8154, 1)

In [275]:
y_train.sum()

433.0

## Build model

In [277]:
# define the  model
def build_model():
  model = Sequential()
  model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
  model.add(Dense(64,activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # compile the keras model
  # opt = tf.keras.optimizers.Adam(clipnorm=1, learning_rate=0.01)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [280]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               8576      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 16,897
Trainable params: 16,897
Non-trainable params: 0
_________________________________________________________________


## Training model

In [285]:
## clear previous model.  Make sure training from 0. 
del model
clear_session()
tf.compat.v1.reset_default_graph()
reset_seeds()
model = build_model()

RANDOM SEEDS RESET


In [291]:
model.fit(X_train, y_train, epochs=100,batch_size=64,class_weight={0:4, 1:200})

Epoch 1/100
351/351 [==============================] - 1s 2ms/step - loss: 0.3249 - accuracy: 0.9714
Epoch 2/100
351/351 [==============================] - 1s 2ms/step - loss: 0.4075 - accuracy: 0.9675
Epoch 3/100
351/351 [==============================] - 1s 2ms/step - loss: 0.3662 - accuracy: 0.9682
Epoch 4/100
351/351 [==============================] - 1s 2ms/step - loss: 0.3265 - accuracy: 0.9730
Epoch 5/100
351/351 [==============================] - 1s 2ms/step - loss: 0.3822 - accuracy: 0.9690
Epoch 6/100
351/351 [==============================] - 1s 2ms/step - loss: 0.3812 - accuracy: 0.9680
Epoch 7/100
351/351 [==============================] - 1s 2ms/step - loss: 0.3458 - accuracy: 0.9714
Epoch 8/100
351/351 [==============================] - 1s 2ms/step - loss: 0.3439 - accuracy: 0.9728
Epoch 9/100
351/351 [==============================] - 1s 2ms/step - loss: 0.4057 - accuracy: 0.9666
Epoch 10/100
351/351 [==============================] - 1s 2ms/step - loss: 0.3348 - accura

In [287]:
loss,accuracy = model.evaluate(X_test, y_test)
print(loss, accuracy)

255/255 [==============================] - 0s 1ms/step - loss: 0.7816 - accuracy: 0.9587
0.7816275954246521 0.9586706161499023


## Predict on eval dataset for further labeling.

In [292]:
y_pred = model.predict(X_test)
y_cls = (y_pred > 0.5).astype("int32")

In [60]:
(y_pred > 0.5).sum()

827

In [289]:
def eval_model(y_test, y_cls):
  y1 = y_test.reshape(-1)
  y2 = y_cls.reshape(-1)
  print(roc_auc_score(y1, y2))
  print(classification_report(y1,y2))
  print(confusion_matrix(y1,y2))


In [284]:
eval_model(y_train, y_cls)

0.5355162712576108
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98     22000
         1.0       0.09      0.09      0.09       433

    accuracy                           0.97     22433
   macro avg       0.54      0.54      0.54     22433
weighted avg       0.96      0.97      0.97     22433

[[21632   368]
 [  395    38]]


In [203]:
eval_model(y_train, y_cls)

0.5064665001502765
              precision    recall  f1-score   support

         0.0       0.99      0.67      0.80     36799
         1.0       0.01      0.35      0.02       433

    accuracy                           0.66     37232
   macro avg       0.50      0.51      0.41     37232
weighted avg       0.98      0.66      0.79     37232

[[24527 12272]
 [  283   150]]


In [290]:
eval_model(y_test, y_cls)

0.6254853896103897
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      8000
         1.0       0.16      0.28      0.20       154

    accuracy                           0.96      8154
   macro avg       0.57      0.63      0.59      8154
weighted avg       0.97      0.96      0.96      8154

[[7774  226]
 [ 111   43]]


In [293]:
eval_model(y_test, y_cls)

0.6058798701298701
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      8000
         1.0       0.14      0.24      0.18       154

    accuracy                           0.96      8154
   macro avg       0.56      0.61      0.58      8154
weighted avg       0.97      0.96      0.96      8154

[[7772  228]
 [ 117   37]]


In [84]:
def export_result(df_eval,y_pred):
  '''
  Attach the predicte result (probability) into original records.
  Export to a csv file for further investication
  '''
  df_eval['predict'] = y_pred
  df_eval['mid_value'] = abs(df_eval['predict'] - 0.5)
  df_eval = df_eval.sort_values('mid_value')
  value_limit = df_eval.iloc[200,-1]
  df_record4label = df_eval[(df_eval['mid_value'] <= value_limit)|(df_eval['fraud'] == 1)]
  path = '/content/drive/MyDrive/Colab Notebooks/finalproject/'
  file_csv = path + 'for_label.csv'
  df_record4label.to_csv(file_csv)
  return True


In [85]:
record_to_label(df_eval,y_pred)

True